In [3]:
# 模型搭建

from datacleanv2 import *
import pyreadstat
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence, pad_packed_sequence
from collections import Counter # 用于统计计数的工具
import time # 用于计时
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.rnn as rnn_utils # 用于处理变长序列，如填充和打包
from torch.utils.data import Dataset, DataLoader # PyTorch 数据加载工具
import pandas as pd
import numpy as np
from tqdm import tqdm # 进度条库，使用 tqdm.tqdm
import random
import copy # 用于复制模型参数或列表
import matplotlib.pyplot as plt # 用于绘图
import seaborn as sns # 用于更美观的统计图，特别是热力图
# 设置中文字体，解决可视化乱码问题
plt.rcParams['font.family'] = ['SimHei'] # 使用黑体，或其他支持中文的字体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像时负号'-'显示为方块的问题


In [5]:
df,mata = pyreadstat.read_sav(r"E:\复旦大学\研一上\科研\评分剪枝算法\数据\data\CBA_cp025q01_logs12_SPSS.sav")
final_result_list_raw=create_time_series_with_result(remove_consecutive_duplicates(transform_sub_dfs(split_strict_paired_events(df))))
#final_result_list = random.sample(final_result_list_raw, 10000)
transformed_list_mix = transform_setting_data_handle_categorical_input(final_result_list_raw)
transformed_list=[item for item in transformed_list_mix if len(item[0])>=3]#去除过短的序列

In [6]:
total_sequences = len(transformed_list)
long_enough_indices = np.where(np.array([len(item[0]) for item in transformed_list]) >= 3)[0]
print(f"初始随机分配到主簇 (长度>=3 的序列): {len(long_enough_indices)}")
print(f"初始未分配序列 (长度<3): {total_sequences - len(long_enough_indices)}")
active_indices_mask = np.array([len(item[0]) for item in transformed_list]) >= 3 
active_indices = np.where(active_indices_mask)[0]
current_total_active_sequences = len(active_indices)

初始随机分配到主簇 (长度>=3 的序列): 23669
初始未分配序列 (长度<3): 0


In [29]:
np.where(active_indices_mask)#np.where返回的是元组，取第一个元素才是索引数组

(array([    0,     1,     2, ..., 23666, 23667, 23668]),)

In [ ]:
np.where(np.array([[[1,2,3],[4,5,6]]])
         >3)

(array([0, 0, 0]), array([1, 1, 1]), array([0, 1, 2]))